In [306]:
import pandas as pd
import wbgapi as wb
import numpy as np
import statsmodels.api as sm
import yfinance as yf

import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.formula.api import logit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

# Q1

In [307]:
Literacy=wb.data.DataFrame('SE.ADT.LITR.FE.ZS','TUR')
Labor=wb.data.DataFrame('SL.TLF.TOTL.FE.ZS','TUR')
Poverty=wb.data.DataFrame('SI.POV.NAHC','TUR')
health=wb.data.DataFrame('SH.XPD.CHEX.PC.CD','TUR')
GDP=wb.data.DataFrame('NY.GDP.PCAP.CD','TUR')
Mortality=wb.data.DataFrame('SH.DYN.MORT','TUR')

In [308]:
df=pd.concat([Mortality.transpose(),Literacy.transpose(),Labor.transpose(),Poverty.transpose(),health.transpose(),GDP.transpose()],axis=1)

In [309]:
df.columns=['Mortality','Literacy','Labor','Poverty','health','GDP']

In [310]:
model = ols('Mortality ~ Literacy + Labor + Poverty + health + GDP', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              Mortality   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     169.6
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           3.81e-07
Time:                        23:50:39   Log-Likelihood:                -7.8492
No. Observations:                  13   AIC:                             27.70
Df Residuals:                       7   BIC:                             31.09
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     69.8438      9.987      6.994      0.0

C:\Users\Engin\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Although literacy is the best variable in explaining the model according to sum suquare, we cannot be sure of the sign of the variable when we look at the 95% confidence interval.

In [311]:
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
Literacy,1.0,296.579278,296.579278,815.327082,1.661736e-08
Labor,1.0,8.414644,8.414644,23.132725,1.944516e-03
Poverty,1.0,0.940111,0.940111,2.584462,1.519524e-01
health,1.0,1.829951,1.829951,5.030724,5.981897e-02
GDP,1.0,0.769732,0.769732,2.116072,1.890863e-01
Residual,7.0,2.546285,0.363755,NaN,NaN


In [312]:
model = ols('Mortality ~ Literacy * Labor ', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              Mortality   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     392.7
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           1.83e-11
Time:                        23:50:41   Log-Likelihood:                -26.882
No. Observations:                  15   AIC:                             61.76
Df Residuals:                      11   BIC:                             64.60
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       -288.5163    107.135     -2.

C:\Users\Engin\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Literacy looks okey but residual is higher than other model. So we might need other variables too.

In [313]:
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
Literacy,1.0,2724.493944,2724.493944,947.190065,5.048734e-12
Labor,1.0,616.218427,616.218427,214.232802,1.474837e-08
Literacy:Labor,1.0,47.884604,47.884604,16.647430,1.819812e-03
Residual,11.0,31.640359,2.876396,NaN,NaN


In [314]:
model = ols('Mortality ~ Literacy * Labor * health ', data=df).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:              Mortality   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     426.1
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           1.16e-07
Time:                        23:50:44   Log-Likelihood:               -0.39099
No. Observations:                  14   AIC:                             16.78
Df Residuals:                       6   BIC:                             21.89
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              1075.47

C:\Users\Engin\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


,df,sum_sq,mean_sq,F,PR(>F)
Literacy,1.0,412.311533,412.311533,2854.074537,2.887445e-09
Labor,1.0,4.376534,4.376534,30.294945,1.509113e-03
Literacy:Labor,1.0,0.842607,0.842607,5.832633,5.221123e-02
health,1.0,8.099188,8.099188,56.063643,2.932256e-04
Literacy:health,1.0,4.604292,4.604292,31.871515,1.324424e-03
Labor:health,1.0,0.267232,0.267232,1.849812,2.226863e-01
Literacy:Labor:health,1.0,0.388972,0.388972,2.692515,1.519300e-01
Residual,6.0,0.866785,0.144464,NaN,NaN


## Q1 result

R2 score is pretty high. Variables explain model very good and residual is small enough. According to %95 confidence intervals variables signs are not stable. But i didn't want to remove the variables which describe the model. I just tried to stabilize literacy which is the most important variable that explains model.

In [315]:
model = ols('Mortality ~ Literacy * Labor + health ', data=df).fit()
print(model.summary())
sm.stats.anova_lm(model)

                            OLS Regression Results                            
Dep. Variable:              Mortality   R-squared:                       0.986
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     156.3
Date:                Mon, 07 Nov 2022   Prob (F-statistic):           2.63e-08
Time:                        23:50:46   Log-Likelihood:                -14.081
No. Observations:                  14   AIC:                             38.16
Df Residuals:                       9   BIC:                             41.36
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        -22.9930    132.626     -0.

C:\Users\Engin\anaconda3\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


,df,sum_sq,mean_sq,F,PR(>F)
Literacy,1.0,412.311533,412.311533,605.620000,1.447968e-09
Labor,1.0,4.376534,4.376534,6.428432,3.194917e-02
Literacy:Labor,1.0,0.842607,0.842607,1.237655,2.947548e-01
health,1.0,8.099188,8.099188,11.896418,7.284845e-03
Residual,9.0,6.127281,0.680809,NaN,NaN


# Q2

In [316]:
Gold = yf.download('GC=F')
Silver = yf.download('SI=F')
Copper = yf.download('HG=F')
Platinum = yf.download('PL=F')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [317]:
Gold_close=Gold['Close']
Silver_open=Silver['Open']
Copper_open=Copper['Open']
Platinum_open=Platinum['Open']

In [318]:
df=pd.concat([Gold_close, Silver_open,Copper_open,Platinum_open], axis=1).dropna()
df.columns=['Gold_close','Silver_open','Copper_open','Platinum_open']

In [319]:
df

,Gold_close,Silver_open,Copper_open,Platinum_open
Date,,,,
2000-08-30 00:00:00-04:00,273.899994,4.950000,0.8790,593.900024
2000-08-31 00:00:00-04:00,278.299988,4.920000,0.8850,589.000000
2000-09-01 00:00:00-04:00,277.000000,5.035000,0.8780,588.000000
2000-09-05 00:00:00-04:00,275.799988,4.990000,0.8960,602.000000
2000-09-06 00:00:00-04:00,274.200012,5.000000,0.9050,603.000000
...,...,...,...,...
2022-11-01 00:00:00-04:00,1645.000000,19.125000,3.4945,959.799988
2022-11-02 00:00:00-04:00,1645.699951,19.780001,3.4985,960.200012
2022-11-03 00:00:00-04:00,1627.300049,19.235001,3.4455,933.400024


In [320]:
model = ols('Gold_close ~ Silver_open * Copper_open * Platinum_open', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             Gold_close   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                 1.529e+04
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:50:50   Log-Likelihood:                -29674.
No. Observations:                4865   AIC:                         5.936e+04
Df Residuals:                    4857   BIC:                         5.942e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [321]:
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
Silver_open,1.0,9.856183e+08,9.856183e+08,84636.959645,0.000000e+00
Copper_open,1.0,3.639321e+07,3.639321e+07,3125.155743,0.000000e+00
Silver_open:Copper_open,1.0,7.049376e+07,7.049376e+07,6053.435763,0.000000e+00
Platinum_open,1.0,1.315517e+08,1.315517e+08,11296.598232,0.000000e+00
Silver_open:Platinum_open,1.0,1.707117e+07,1.707117e+07,1465.934747,1.547806e-280
Copper_open:Platinum_open,1.0,3.916590e+06,3.916590e+06,336.325187,1.089672e-72
Silver_open:Copper_open:Platinum_open,1.0,1.213660e+06,1.213660e+06,104.219369,3.176283e-24
Residual,4857.0,5.656097e+07,1.164525e+04,NaN,NaN


### Silver closing price

In [322]:
Silver_close=Silver['Close']
Gold_open=Gold['Open']
Copper_open=Copper['Open']
Platinum_open=Platinum['Open']
df=pd.concat([Silver_close, Gold_open,Copper_open,Platinum_open], axis=1).dropna()
df.columns=['Silver_close','Gold_open','Copper_open','Platinum_open']

In [323]:
model = ols('Silver_close ~ Gold_open * Copper_open * Platinum_open', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           Silver_close   R-squared:                       0.972
Model:                            OLS   Adj. R-squared:                  0.972
Method:                 Least Squares   F-statistic:                 2.431e+04
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:50:58   Log-Likelihood:                -8683.6
No. Observations:                4865   AIC:                         1.738e+04
Df Residuals:                    4857   BIC:                         1.744e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [324]:
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
Gold_open,1.0,275713.050883,275713.050883,132390.617583,0.000000e+00
Copper_open,1.0,20673.529903,20673.529903,9926.919973,0.000000e+00
Gold_open:Copper_open,1.0,3740.642554,3740.642554,1796.164441,0.000000e+00
Platinum_open,1.0,38477.292015,38477.292015,18475.848120,0.000000e+00
Gold_open:Platinum_open,1.0,11323.812098,11323.812098,5437.415720,0.000000e+00
Copper_open:Platinum_open,1.0,104.354998,104.354998,50.108700,1.662265e-12
Gold_open:Copper_open:Platinum_open,1.0,4399.749779,4399.749779,2112.651500,0.000000e+00
Residual,4857.0,10115.054319,2.082572,NaN,NaN


### Copper closing price

In [325]:
Copper_close=Copper['Close']
Gold_open=Gold['Open']
Silver_open=Silver['Open']
Platinum_open=Platinum['Open']
df=pd.concat([Copper_close, Gold_open,Copper_open,Platinum_open], axis=1).dropna()
df.columns=['Copper_close','Gold_open','Silver_open','Platinum_open']

In [326]:
model = ols('Copper_close ~ Gold_open * Silver_open * Platinum_open', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           Copper_close   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 5.903e+05
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:51:01   Log-Likelihood:                 8991.4
No. Observations:                4865   AIC:                        -1.797e+04
Df Residuals:                    4857   BIC:                        -1.791e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [327]:
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
Gold_open,1.0,4140.915989,4140.915989,2.845553e+06,0.000000
Silver_open,1.0,1872.428729,1872.428729,1.286695e+06,0.000000
Gold_open:Silver_open,1.0,0.005467,0.005467,3.756896e+00,0.052648
Platinum_open,1.0,0.000270,0.000270,1.855658e-01,0.666652
Gold_open:Platinum_open,1.0,0.018405,0.018405,1.264759e+01,0.000380
Silver_open:Platinum_open,1.0,0.000624,0.000624,4.288173e-01,0.512601
Gold_open:Silver_open:Platinum_open,1.0,0.000624,0.000624,4.285132e-01,0.512750
Residual,4857.0,7.068020,0.001455,NaN,NaN


### Platinum closing price

In [328]:
Platinum_close=Platinum['Close']
Gold_open=Gold['Open']
Silver_open=Silver['Open']
Copper_open=Copper['Open']
df=pd.concat([Platinum_close, Gold_open,Copper_open,Silver_open], axis=1).dropna()
df.columns=['Platinum_close','Gold_open','Copper_open','Silver_open']

In [329]:
model = ols('Platinum_close ~ Gold_open * Silver_open * Copper_open', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         Platinum_close   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.871
Method:                 Least Squares   F-statistic:                     4695.
Date:                Mon, 07 Nov 2022   Prob (F-statistic):               0.00
Time:                        23:51:03   Log-Likelihood:                -30636.
No. Observations:                4865   AIC:                         6.129e+04
Df Residuals:                    4857   BIC:                         6.134e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

In [330]:
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
Gold_open,1.0,1.636205e+08,1.636205e+08,9461.506034,0.000000e+00
Silver_open,1.0,2.833683e+08,2.833683e+08,16386.033275,0.000000e+00
Gold_open:Silver_open,1.0,9.575900e+07,9.575900e+07,5537.351541,0.000000e+00
Copper_open,1.0,1.427727e+07,1.427727e+07,825.596345,8.037405e-168
Gold_open:Copper_open,1.0,1.111405e+06,1.111405e+06,64.267993,1.349494e-15
Silver_open:Copper_open,1.0,6.507264e+06,6.507264e+06,376.288475,8.508546e-81
Gold_open:Silver_open:Copper_open,1.0,3.677464e+06,3.677464e+06,212.652718,3.554063e-47
Residual,4857.0,8.399349e+07,1.729329e+04,NaN,NaN


## Result

The model in which we examined the closing price of copper is seen as the best. 𝑅2 score higher than others. The difference between the coefficients of the independent variables according to the 95% confidence intervals is not much and they have the same signs.

# Q3

In [331]:
def get_ticker(data):
    lst=[]
    data1 = yf.download(data)
    data1=data1[['Open','Close']]
    for i in data1.index:
        if data1.loc[i]['Close']>data1.loc[i]['Open']:
            lst.append(1)
        else:
            lst.append(0)
    data1['result']=lst
    return data1

In [332]:
Gold=get_ticker('GC=F')
silver_open=get_ticker('SI=F')['Open']
copper_open=get_ticker('HG=F')['Open']
platinum_open=get_ticker('PL=F')['Open']

[*********************100%***********************]  1 of 1 completed


C:\Users\Engin\AppData\Local\Temp/ipykernel_1816/3116983416.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['result']=lst


[*********************100%***********************]  1 of 1 completed


C:\Users\Engin\AppData\Local\Temp/ipykernel_1816/3116983416.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['result']=lst


[*********************100%***********************]  1 of 1 completed


C:\Users\Engin\AppData\Local\Temp/ipykernel_1816/3116983416.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['result']=lst


[*********************100%***********************]  1 of 1 completed


C:\Users\Engin\AppData\Local\Temp/ipykernel_1816/3116983416.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['result']=lst


In [333]:
df3=pd.concat([Gold, Silver_open,copper_open,platinum_open], axis=1).dropna()
df3.columns=['Gold_open','Gold_close','Gold_result','Silver_open','Copper_open','Platinum_open']

In [334]:
df3

,Gold_open,Gold_close,Gold_result,Silver_open,Copper_open,Platinum_open
Date,,,,,,
2000-08-30 00:00:00-04:00,273.899994,273.899994,0.0,4.950000,0.8790,593.900024
2000-08-31 00:00:00-04:00,274.799988,278.299988,1.0,4.920000,0.8850,589.000000
2000-09-01 00:00:00-04:00,277.000000,277.000000,0.0,5.035000,0.8780,588.000000
2000-09-05 00:00:00-04:00,275.799988,275.799988,0.0,4.990000,0.8960,602.000000
2000-09-06 00:00:00-04:00,274.200012,274.200012,0.0,5.000000,0.9050,603.000000
...,...,...,...,...,...,...
2022-11-01 00:00:00-04:00,1630.800049,1645.000000,1.0,19.125000,3.4945,959.799988
2022-11-02 00:00:00-04:00,1650.800049,1645.699951,0.0,19.780001,3.4985,960.200012
2022-11-03 00:00:00-04:00,1629.199951,1627.300049,0.0,19.235001,3.4455,933.400024


In [335]:
model = logit('Gold_result ~ Gold_open * Silver_open * Copper_open * Platinum_open', data=df3).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.666791
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Gold_result   No. Observations:                 4865
Model:                          Logit   Df Residuals:                     4849
Method:                           MLE   Df Model:                           15
Date:                Mon, 07 Nov 2022   Pseudo R-squ.:                 0.02221
Time:                        23:51:14   Log-Likelihood:                -3243.9
converged:                       True   LL-Null:                       -3317.6
Covariance Type:            nonrobust   LLR p-value:                 7.910e-24
===================================================================================================================
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                                          -2.6255      1.317     -1.993      0.046      -5.208      -0.043
Gold_open                                           0.0090      0.003      3.166      0.002       0.003       0.015
Silver_open                                        -0.3046      0.385     -0.790      0.429      -1.060       0.451
Gold_open:Silver_open                            6.458e-05      0.000      0.273      0.785      -0.000       0.001
Copper_open                                        -1.9343      1.060     -1.825      0.068      -4.011       0.143
Gold_open:Copper_open                              -0.0009      0.001     -0.850      0.395      -0.003       0.001
Silver_open:Copper_open                             0.2034      0.107      1.893      0.058      -0.007       0.414
Gold_open:Silver_open:Copper_open               -6.899e-05   6.72e-05     -1.026      0.305      -0.000    6.28e-05
Platinum_open                                       0.0035      0.002      1.906      0.057   -9.88e-05       0.007
Gold_open:Platinum_open                         -1.299e-05   2.94e-06     -4.423      0.000   -1.87e-05   -7.23e-06
Silver_open:Platinum_open                           0.0006      0.000      1.566      0.117      -0.000       0.001
Gold_open:Silver_open:Platinum_open             -1.148e-07   2.22e-07     -0.517      0.605    -5.5e-07     3.2e-07
Copper_open:Platinum_open                           0.0003      0.001      0.436      0.663      -0.001       0.002
Gold_open:Copper_open:Platinum_open              2.343e-06   9.12e-07      2.569      0.010    5.56e-07    4.13e-06
Silver_open:Copper_open:Platinum_open              -0.0002   9.62e-05     -2.069      0.039      -0.000   -1.05e-05
Gold_open:Silver_open:Copper_open:Platinum_open  4.832e-08   5.46e-08      0.885      0.376   -5.87e-08    1.55e-07
===================================================================================================================
"""

### Silver closing price

In [336]:
Gold_open=Gold['Open']
silver=get_ticker('SI=F')

[*********************100%***********************]  1 of 1 completed


C:\Users\Engin\AppData\Local\Temp/ipykernel_1816/3116983416.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['result']=lst


In [337]:
df3=pd.concat([silver, Gold_open,copper_open,platinum_open], axis=1).dropna()
df3.columns=['silver_open','silver_close','silver_result','Gold_open','Copper_open','Platinum_open']

In [338]:
model = logit('silver_result ~ Gold_open * silver_open * Copper_open * Platinum_open', data=df3).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.627076
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          silver_result   No. Observations:                 4865
Model:                          Logit   Df Residuals:                     4849
Method:                           MLE   Df Model:                           15
Date:                Mon, 07 Nov 2022   Pseudo R-squ.:                 0.01987
Time:                        23:51:20   Log-Likelihood:                -3050.7
converged:                       True   LL-Null:                       -3112.6
Covariance Type:            nonrobust   LLR p-value:                 3.652e-19
===================================================================================================================
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                                          -5.0543      1.380     -3.663      0.000      -7.759      -2.350
Gold_open                                           0.0082      0.003      2.823      0.005       0.002       0.014
silver_open                                         0.2209      0.402      0.550      0.582      -0.566       1.008
Gold_open:silver_open                              -0.0002      0.000     -0.999      0.318      -0.001       0.000
Copper_open                                         1.9374      1.101      1.760      0.078      -0.220       4.095
Gold_open:Copper_open                              -0.0024      0.001     -2.169      0.030      -0.005      -0.000
silver_open:Copper_open                            -0.1012      0.113     -0.899      0.369      -0.322       0.119
Gold_open:silver_open:Copper_open                8.044e-05   6.99e-05      1.151      0.250   -5.66e-05       0.000
Platinum_open                                       0.0041      0.002      2.146      0.032       0.000       0.008
Gold_open:Platinum_open                         -9.574e-06   3.03e-06     -3.156      0.002   -1.55e-05   -3.63e-06
silver_open:Platinum_open                          -0.0002      0.000     -0.373      0.710      -0.001       0.001
Gold_open:silver_open:Platinum_open              2.814e-07   2.32e-07      1.215      0.224   -1.73e-07    7.35e-07
Copper_open:Platinum_open                          -0.0020      0.001     -2.846      0.004      -0.003      -0.001
Gold_open:Copper_open:Platinum_open              2.879e-06   9.32e-07      3.088      0.002    1.05e-06    4.71e-06
silver_open:Copper_open:Platinum_open            9.141e-05      0.000      0.911      0.363      -0.000       0.000
Gold_open:silver_open:Copper_open:Platinum_open -9.477e-08   5.68e-08     -1.670      0.095   -2.06e-07    1.65e-08
===================================================================================================================
"""

### Copper closing price

In [339]:
copper=get_ticker('HG=F')

[*********************100%***********************]  1 of 1 completed


C:\Users\Engin\AppData\Local\Temp/ipykernel_1816/3116983416.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['result']=lst


In [340]:
df3=pd.concat([copper, Gold_open,silver_open,platinum_open], axis=1).dropna()
df3.columns=['copper_open','copper_close','copper_result','Gold_open','silver_open','Platinum_open']

In [341]:
model = logit('copper_result ~ Gold_open * silver_open * copper_open * Platinum_open', data=df3).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.687446
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          copper_result   No. Observations:                 4865
Model:                          Logit   Df Residuals:                     4849
Method:                           MLE   Df Model:                           15
Date:                Mon, 07 Nov 2022   Pseudo R-squ.:                0.007940
Time:                        23:51:23   Log-Likelihood:                -3344.4
converged:                       True   LL-Null:                       -3371.2
Covariance Type:            nonrobust   LLR p-value:                 3.141e-06
===================================================================================================================
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                                          -4.3400      1.286     -3.375      0.001      -6.860      -1.819
Gold_open                                           0.0075      0.003      2.696      0.007       0.002       0.013
silver_open                                         0.0183      0.381      0.048      0.962      -0.729       0.766
Gold_open:silver_open                           -8.315e-05      0.000     -0.354      0.723      -0.001       0.000
copper_open                                         1.6178      1.032      1.568      0.117      -0.405       3.641
Gold_open:copper_open                              -0.0024      0.001     -2.236      0.025      -0.004      -0.000
silver_open:copper_open                            -0.0275      0.106     -0.260      0.795      -0.235       0.180
Gold_open:silver_open:copper_open                 4.61e-05   6.66e-05      0.692      0.489   -8.44e-05       0.000
Platinum_open                                       0.0058      0.002      3.294      0.001       0.002       0.009
Gold_open:Platinum_open                         -1.094e-05   2.88e-06     -3.804      0.000   -1.66e-05   -5.31e-06
silver_open:Platinum_open                        5.732e-05      0.000      0.142      0.887      -0.001       0.001
Gold_open:silver_open:Platinum_open              1.673e-07    2.2e-07      0.762      0.446   -2.63e-07    5.98e-07
copper_open:Platinum_open                          -0.0020      0.001     -3.010      0.003      -0.003      -0.001
Gold_open:copper_open:Platinum_open               3.08e-06   9.01e-07      3.417      0.001    1.31e-06    4.85e-06
silver_open:copper_open:Platinum_open            1.946e-05   9.51e-05      0.205      0.838      -0.000       0.000
Gold_open:silver_open:copper_open:Platinum_open -6.614e-08   5.43e-08     -1.219      0.223   -1.73e-07    4.02e-08
===================================================================================================================
"""

### Platinum closing price

In [342]:
platinum=get_ticker('PL=F')

[*********************100%***********************]  1 of 1 completed


C:\Users\Engin\AppData\Local\Temp/ipykernel_1816/3116983416.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['result']=lst


In [343]:
df3=pd.concat([platinum, Gold_open,silver_open,copper_open], axis=1).dropna()
df3.columns=['platinum_open','platinum_close','platinum_result','Gold_open','silver_open','copper_open']

In [344]:
model = logit('platinum_result ~ Gold_open * silver_open * copper_open * platinum_open', data=df3).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.526446
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        platinum_result   No. Observations:                 4865
Model:                          Logit   Df Residuals:                     4849
Method:                           MLE   Df Model:                           15
Date:                Mon, 07 Nov 2022   Pseudo R-squ.:                 0.09111
Time:                        23:51:26   Log-Likelihood:                -2561.2
converged:                       True   LL-Null:                       -2817.9
Covariance Type:            nonrobust   LLR p-value:                7.852e-100
===================================================================================================================
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                                          -3.3828      1.503     -2.251      0.024      -6.329      -0.437
Gold_open                                           0.0083      0.004      2.306      0.021       0.001       0.015
silver_open                                         0.4692      0.461      1.017      0.309      -0.435       1.374
Gold_open:silver_open                              -0.0007      0.000     -2.461      0.014      -0.001      -0.000
copper_open                                         0.0773      1.217      0.064      0.949      -2.308       2.463
Gold_open:copper_open                              -0.0031      0.001     -2.188      0.029      -0.006      -0.000
silver_open:copper_open                            -0.0849      0.124     -0.687      0.492      -0.327       0.157
Gold_open:silver_open:copper_open                   0.0002   8.12e-05      2.899      0.004    7.63e-05       0.000
platinum_open                                       0.0036      0.002      1.850      0.064      -0.000       0.007
Gold_open:platinum_open                         -6.142e-06    3.6e-06     -1.707      0.088   -1.32e-05     9.1e-07
silver_open:platinum_open                          -0.0005      0.000     -1.005      0.315      -0.001       0.000
Gold_open:silver_open:platinum_open              5.697e-07   2.64e-07      2.155      0.031    5.16e-08    1.09e-06
copper_open:platinum_open                          -0.0001      0.001     -0.167      0.868      -0.002       0.001
Gold_open:copper_open:platinum_open               1.19e-06   1.18e-06      1.007      0.314   -1.13e-06    3.51e-06
silver_open:copper_open:platinum_open               0.0001      0.000      1.067      0.286      -0.000       0.000
Gold_open:silver_open:copper_open:platinum_open -1.741e-07    6.8e-08     -2.559      0.010   -3.07e-07   -4.08e-08
===================================================================================================================
"""

##  Q3 Result

when we compare all models, the platinum model I built last is the best. R2 score is higher than others and has more consistant variables. But it is not a good model either. R2 is very low. independent variables fail to explain the model.

# Q4

In [345]:
credit = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', header=None)

fn = {'+': 1, '-': 0}

X = credit.replace('?',0).iloc[:,[1,2,7,10,14]]
y = credit.iloc[:,15].map(lambda x: fn.get(x,0))

In [346]:
credit

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


In [347]:
df4=pd.concat([y,X],axis=1)

In [348]:
df4.columns=['onbes','bir','iki','yedi','on','ondort']

In [349]:
df4.corr()

,onbes,iki,yedi,on,ondort
onbes,1.000000,0.206294,0.322475,0.406410,0.175657
iki,0.206294,1.000000,0.298902,0.271207,0.123121
yedi,0.322475,0.298902,1.000000,0.322330,0.051345
on,0.406410,0.271207,0.322330,1.000000,0.063692
ondort,0.175657,0.123121,0.051345,0.063692,1.000000


In [350]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75)
model = LogisticRegression(max_iter=1500)
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
confusion_matrix(y_test,y_predict)

array([[85, 12],
       [29, 47]], dtype=int64)

In [351]:
def bootstrap(X,y,model):
    res = []
    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75)
        model.fit(X_train,y_train)
        res.append(model.score(X_test,y_test))
    tmp = sorted(res)[3:97]
    return (min(tmp),max(tmp))

In [352]:
bootstrap(X,y,model)

(0.7052023121387283, 0.8208092485549133)